In [63]:
import whisper
import pyaudio
import wave
import os
from pyChatGPT import ChatGPT
from time import sleep
from gtts import gTTS
import mutagen
import sounddevice as sd
import soundfile as sf

In [64]:
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
RECORD_SECONDS = 7
WAVE_OUTPUT_FILENAME = "audio.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()



* recording
* done recording


In [65]:
model = whisper.load_model("base")
result = model.transcribe("audio.wav")
result_text = result["text"]
language = result['language']
print(result['text'])
os.remove("audio.wav")

C:\Users\András\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\whisper\transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Write me a short poem about summer.


In [66]:
print(result)

{'text': ' Write me a short poem about summer.', 'segments': [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 7.0, 'text': ' Write me a short poem about summer.', 'tokens': [50364, 23499, 385, 257, 2099, 13065, 466, 4266, 13, 50714], 'temperature': 0.0, 'avg_logprob': -0.46199503811922943, 'compression_ratio': 0.813953488372093, 'no_speech_prob': 0.11185072362422943}], 'language': 'en'}


In [67]:
session_token = ''
api = ChatGPT(session_token)
resp = api.send_message(result_text)
resp_txt = resp['message']
print(resp['message'])



Summer sun, warm and bright,
Bringing life to all in sight.
Fields of green and flowers in bloom,
Nature's beauty on full display, soon.
Birds singing sweet melodies,
Breezes blowing gently, a pleasant breeze.
Children playing, laughter in the air,
Summertime, a season without a care.
So let us soak up the sun,
And enjoy all that summer has begun.




In [68]:
myobj = gTTS(text=resp_txt, lang=language, slow=False)
myobj.save("response.wav")

filename = 'response.wav'
# Extract data and sampling rate from file
data, fs = sf.read(filename, dtype='float32')  
sd.play(data, fs)
status = sd.wait()  # Wait until file is done playing
os.remove("response.wav")